# Filter curves

This notebook shows how you can apply different filters to your rubix IFU cube and create photometric images of your mock-data.

In [ ]:
# NBVAL_SKIP
from rubix.telescope.filters import load_filter, print_filter_list, print_filter_list_info, print_filter_property

## Information about the filters

We can have a look, which different filters are availible for a given facility or instrument. A list of all availible filters can be found here: http://svo2.cab.inta-csic.es/theory/fps/index.php

As an example, we print the different filters for SLOAN.

In [ ]:
# NBVAL_SKIP
print_filter_list("SLOAN")

We can also print some more details about the filters. `print_filter_list_info()` prints the filter name, the dtype and the unit.

In [ ]:
# NBVAL_SKIP
print_filter_list_info("SLOAN")

The most detaield information about a filter can be obtained by using the `print_filter_property()` function.

In [ ]:
# NBVAL_SKIP
print_filter_property("SLOAN", "SDSS.u")

In [ ]:
# NBVAL_SKIP
print_filter_property("JWST", "F070W", "NIRCam")

## Loading filters

Now we can load and plot our selected filters, in our example case `"SLOAN"`.
If you want to know more about filters and which ones are supported by RUBIX please visit the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/theory/fps/index.php). RUBIX supports all standard filters for all instruments of all facilities listed there. 

In [ ]:
# NBVAL_SKIP
# load all fliter curves for SLOAN
curves = load_filter("SLOAN")

In [ ]:
# NBVAL_SKIP
curves.filters

In [ ]:
# NBVAL_SKIP
curves.plot()

In [ ]:
# NBVAL_SKIP
filter = curves[1]
filter.plot()

## Applying filters to mock-IFUs

After getting the information about different filters and loading the filter curves for `"SLOAN"`, we want to apply these filter curves to a mock-IFU cube to get photometric images of the mock-IFU cube.

The first step is to create our mock-IFU cube. We have taken care of this already and run RUBIX with default `config` for a tiny mock MUSE cube on an example Ilustris TNG galaxy. For more details see `rubix_pipeline_single_function.ipynb` or `rubix_pipeline_stepwise.ipynb`. Below we load the dummy datacube using the library `h5py`.

In [ ]:
#NBVAL_SKIP
import h5py
import numpy as np
with h5py.File('./data/dummy_datacube.h5', 'r') as hf2:
    print(hf2.keys())
    datacube = np.array(hf2.get('datacube'))
    wave = np.array(hf2.get('wave'))

Our dummy datacube has 25x25 pixels and 3721 spectral bins.

In [ ]:
# NBVAL_SKIP
datacube.shape

Now, we have our mock-IFU datacube and we have selected and loaded a filter. The next step is to apply the filter to the datacube, which is done with a convolution. And then we obtain our photometric image of the galaxy. For the filter, choosen in this example, you may wonder, why the image is zerro everywhere. You have to keep in mind that our dummy datacube is created for a MUSE observation and in the default `telescopes.yaml` we defined the wavelength to be in the range `[4700.15, 9351.4]`and the filter is in the range `[3000, 4000]`. So this result should be expected for the choice of this mock-data convolved with the `SLOAN/SDSS.u`filter.

In [ ]:
# NBVAL_SKIP
from rubix.telescope.filters import convolve_filter_with_spectra
import matplotlib.pyplot as plt

In [ ]:
# NBVAL_SKIP
filter = curves[1]

In [ ]:
# NBVAL_SKIP
convolved = convolve_filter_with_spectra(filter, datacube, wave)
print(convolved.shape)

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
plt.imshow(convolved)
plt.colorbar()

If we now look at other filters from `SLOAN/SDSS`that match the wavelengthrange of our mock-datacube, we get photometric images of our galaxy.

In [ ]:
# NBVAL_SKIP
for filter in curves:
    convolved = convolve_filter_with_spectra(filter, datacube, wave)
    plt.figure()
    plt.imshow(convolved)
    plt.colorbar()
    plt.title(filter.name)

In [ ]:
# NBVAL_SKIP
filters,images =curves.apply_filter_curves(datacube, wave).values()

In [ ]:
# NBVAL_SKIP
filters

In [ ]:
# NBVAL_SKIP
for i,name in zip(images, filters):
    plt.figure()
    plt.imshow(i)
    plt.colorbar()
    plt.title(name)

To create false color images (RGB images), we have to normalize the individual photometric images from three different filters and stack them.

In [ ]:
# NBVAL_SKIP
# Create an RGB image
# Normalize the images
import numpy as np

def normalize(image):
    image_min = image.min()
    image_max = image.max()
    return (image - image_min) / (image_max - image_min)

r = images[1]
g = images[2]
b = images[3]

rgb = np.stack([r,g,b], axis=-1)

rgb = normalize(rgb)

plt.imshow(rgb)

